In [5]:
import os
os.chdir("/root/data/DBP_sa_bc/")
from os.path import join as pj
import argparse
import sys
sys.path.append("modules")
import utils
import numpy as np
import anndata as ad
import pandas as pd
import copy

In [6]:
parser = argparse.ArgumentParser()
# parser.add_argument('--tasks', type=str, nargs='+',  default=["dogma_full", "dogma_paired_full", 
#     "dogma_paired_abc", "dogma_paired_ab",  "dogma_paired_ac", "dogma_paired_bc",
#     "dogma_single_full", "dogma_single"])
parser.add_argument('--tasks', type=str, nargs='+',  default=["wnn_rna"])
parser.add_argument('--method', type=str, default='DBP_sa_bc')
# parser.add_argument('--mosaic', type=int, default=1)
parser.add_argument('--experiment', type=str, default='e1')
parser.add_argument('--model', type=str, default='default')
parser.add_argument('--init_model', type=str, default='sp_00001899')
parser.add_argument('--K', type=str, default='50')
o, _ = parser.parse_known_args()  # for python interactive
# o = parser.parse_args()

In [7]:
# DBP_embed results
df_batch_bio_embed = {}
o.tasks = [o.tasks[0]]
for task in o.tasks:
    df_batch_bio_embed[task] = pd.read_excel(pj("result", "comparison", task, o.method, o.experiment, o.init_model, "metrics_batch_bio_break38.xlsx"))
    df_batch_bio_embed[task].rename(index={0: task}, inplace=True)
df_batch_bio_embed_cat = pd.concat(df_batch_bio_embed.values(), axis=0)

df_batch_bio_embed_cat[["task"]] = df_batch_bio_embed_cat.index
df_batch_bio_embed_cat.rename(index={i: "DBP" for i in df_batch_bio_embed_cat.index}, inplace=True)
df_batch_bio_embed_cat


,graph_conn,batch_ASW,kBET,iLISI,label_ASW,il_score_ASW,il_score_f1,NMI,ARI,cLISI,task
DBP,0.948303,0.930286,0.491503,0.550784,0.588353,0.569928,0.82734,0.785458,0.783129,0.998519,wnn_rna


In [8]:
# sota results
methods = [
    "harmony",
    "liger",
    "mofa",
    "scanorama_embed",
    "seurat_cca",
    "seurat_rpca",
    "LDVAE",
    "scETM"
    # "pca",
]

df_sota = {}
for method in methods:
    if "DBP_sa_bc" in method:
        df_sota[method] = pd.read_excel(pj("result", "comparison", o.tasks[0], method, o.experiment, o.init_model, "metrics_batch_bio.xlsx"))
    else:
        df_sota[method] = pd.read_excel(pj("result", "comparison", o.tasks[0], method, o.K, "metrics_batch_bio.xlsx"))
    df_sota[method].rename(index={0: method}, inplace=True)
df_sota_cat = pd.concat(df_sota.values(), axis=0)

# df_sota_cat[["task"]] = o.tasks
# df_sota_cat


In [9]:
df_cat = pd.concat([df_batch_bio_embed_cat, df_sota_cat], axis=0)
df_mean_cat = copy.deepcopy(df_cat)
df_mean_cat["batch_score"] = df_cat[["graph_conn", "batch_ASW", "kBET", "iLISI"]].mean(axis=1)
df_mean_cat["bio_score"] = df_cat[["label_ASW", "il_score_ASW", "il_score_f1", "NMI", "ARI", "cLISI"]].mean(axis=1)
df_mean_cat["overall_score"] = 0.4 * df_mean_cat["batch_score"] + 0.6 * df_mean_cat["bio_score"]
df_mean_cat = df_mean_cat[["graph_conn", "batch_ASW", "kBET", "iLISI", "batch_score", "label_ASW", "il_score_ASW", "il_score_f1", "NMI", "ARI", "cLISI", "bio_score", "overall_score"]]
df_mean_cat.sort_values("overall_score", ascending=False, inplace=True)
df_mean_cat

,graph_conn,batch_ASW,kBET,iLISI,batch_score,label_ASW,il_score_ASW,il_score_f1,NMI,ARI,cLISI,bio_score,overall_score
DBP,0.948303,0.930286,0.491503,0.550784,0.730219,0.588353,0.569928,0.827340,0.785458,0.783129,0.998519,0.758788,0.747360
liger,0.909486,0.920660,0.507055,0.545517,0.720679,0.578310,0.580822,0.800142,0.740996,0.715914,0.998488,0.735779,0.729739
harmony,0.955333,0.924835,0.248445,0.501269,0.657470,0.627489,0.579255,0.834047,0.785969,0.783050,0.998663,0.768079,0.723835
seurat_cca,0.949858,0.908428,0.191318,0.478816,0.632105,0.641669,0.584811,0.808364,0.767616,0.720079,0.999314,0.753642,0.705027
seurat_rpca,0.940861,0.908160,0.144651,0.476937,0.617652,0.643529,0.583395,0.819859,0.743497,0.709848,0.999545,0.749946,0.697028
LDVAE,0.955143,0.938717,0.054850,0.327631,0.569085,0.556199,0.552502,0.860788,0.808495,0.803823,0.998445,0.763375,0.685659
scanorama_embed,0.925924,0.895525,0.146806,0.436456,0.601178,0.550739,0.537712,0.759814,0.722991,0.707736,0.984171,0.710527,0.666787
scETM,0.954486,0.902373,0.066344,0.298947,0.555537,0.640231,0.582806,0.818116,0.727716,0.573797,0.998656,0.723554,0.656347
mofa,0.956750,0.897426,0.015375,0.096984,0.491634,0.566573,0.580909,0.823857,0.683137,0.481318,0.999544,0.689223,0.610187


In [10]:
# df_norm = copy.deepcopy(df)
# df_norm = copy.deepcopy(df)
# for metric in metrics:
#     v = [df[method][metric].values[0] for method in methods]
#     v_min = min(v)
#     v_max = max(v)
#     for method in methods:
#         df_norm[method][metric] = (df[method][metric] - v_min) / (v_max - v_min)
# df_norm_cat = pd.concat(df_norm.values(), axis=0)
# df_norm_cat["batch_score"] = (df_norm_cat["iLISI"] + df_norm_cat["graph_conn"] + df_norm_cat["kBET"]) / 3
# df_norm_cat["bio_score"] = (df_norm_cat["NMI"] + df_norm_cat["ARI"] + df_norm_cat["il_score_f1"] + df_norm_cat["cLISI"]) / 4
# df_norm_cat["overall_score"] = 0.4 * df_norm_cat["batch_score"] + 0.6 * df_norm_cat["bio_score"]
# df_norm_cat = df_norm_cat[["iLISI", "graph_conn", "kBET", "batch_score", "NMI", "ARI", "il_score_f1", "cLISI", "bio_score", "overall_score"]]
# df_norm_cat

In [11]:
out_dir = pj("eval", "plot", "data", o.K)
utils.mkdir(out_dir, remove_old=False)
# df_mean_cat.to_excel(pj(out_dir, "scib_metrics_dbp_"+o.tasks[0].split("_")[0]+"_"+o.init_model+".xlsx"))
df_mean_cat.to_excel(pj(out_dir, "scib_metrics_"+o.tasks[0]+"_"+o.method+"_"+o.experiment+"_"+o.init_model+".xlsx"))